In [1]:
import anndata as ad
import pandas as pd
import torch
from starling import starling, utility
from lightning_lite import seed_everything
import pytorch_lightning as pl

In [2]:
df = pd.read_csv("/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/IMMUcan_quantification.csv")

In [3]:
df.columns

Index(['MPO', 'HistoneH3', 'SMA', 'CD16', 'CD38', 'HLADR', 'CD27', 'CD15',
       'CD45RA', 'CD163', 'B2M', 'CD20', 'CD68', 'Ido1', 'CD3', 'LAG3',
       'CD11c', 'PD1', 'PDGFRb', 'CD7', 'GrzB', 'PDL1', 'TCF7', 'CD45RO',
       'FOXP3', 'ICOS', 'CD8a', 'CarbonicAnhydrase', 'CD33', 'Ki67', 'VISTA',
       'CD40', 'CD4', 'CD14', 'Ecad', 'CD303', 'CD206', 'cleavedPARP', 'DNA1',
       'DNA2', 'image', 'sample_id', 'cell_id', 'x', 'y', 'area',
       'major_axis_length', 'minor_axis_length', 'eccentricity', 'width_px',
       'height_px', 'cell_labels', 'level_1_cell_type', 'level_2_cell_type',
       'cell_type'],
      dtype='object')

In [25]:
X_columns = df.columns[:df.columns.get_loc('image')]
obs_columns =df.columns[df.columns.get_loc('image'):]
adata = ad.AnnData(
    X=df[X_columns],
    obs=df[obs_columns],
    var=pd.DataFrame(index=X_columns)
)

/Users/lukashat/miniforge3/envs/starling/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [26]:
adata.X = adata.X.astype('float32')
adata.obs['area'] = adata.obs['area'].astype('float32')

In [23]:
adata.X

array([[0.        , 1.8199879 , 0.5026498 , ..., 0.42427316, 4.7611303 ,
        5.301351  ],
       [0.1969833 , 2.5416837 , 1.2478923 , ..., 0.05964309, 4.304721  ,
        4.800834  ],
       [0.13592021, 2.3930867 , 1.1369607 , ..., 0.1186204 , 5.029241  ,
        5.5307646 ],
       ...,
       [0.15094134, 4.214958  , 2.651646  , ..., 0.08397473, 4.8224564 ,
        5.3616614 ],
       [0.20371623, 6.0141263 , 2.7575548 , ..., 0.23721097, 4.579855  ,
        5.186302  ],
       [0.4905906 , 5.8223476 , 1.6154104 , ..., 0.22942112, 4.4482117 ,
        5.010813  ]], dtype=float32)

In [27]:
adata = utility.init_clustering("KM", adata, k=10)

In [24]:
adata.obs.dtypes

image                 object
sample_id             object
cell_id                int64
x                    float64
y                    float64
area                   int64
major_axis_length    float64
minor_axis_length    float64
eccentricity         float64
width_px               int64
height_px              int64
cell_labels           object
level_1_cell_type     object
level_2_cell_type     object
cell_type             object
init_label             int32
dtype: object

In [28]:
st = starling.ST(adata)

/Users/lukashat/miniforge3/envs/starling/lib/python3.11/site-packages/starling/starling.py:78: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  torch.tensor(self.adata.obs[self.cell_size_col_name])


In [29]:
log_tb = pl.loggers.TensorBoardLogger(save_dir="log")


In [30]:
cb_early_stopping = pl.callbacks.EarlyStopping(monitor="train_loss", mode="min", verbose=False)

In [31]:
st.train_and_fit(
    callbacks=[cb_early_stopping]
)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name | Type | Params | Mode
-------------------------------------
-------------------------------------
0         Trainable params
0         Non-trainable params
0         Total params
0.000     Total estimated model params size (MB)
0         Modules in train mode
0         Modules in eval mode
/Users/lukashat/miniforge3/envs/starling/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 0:   0%|          | 0/893 [00:00<?, ?it/s] 

TypeError: Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.